In [65]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data = Path("../Resources/schools_complete.csv")
student_data = Path("../Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data)
student_data = pd.read_csv(student_data)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


## Local Government Area Summary

In [66]:
#Calculating the key metrics

#Calculate the total number of umique schools

total_schools = school_data['school_name'].nunique()

# Calculate the total number of students

total_students= school_data_complete['Student ID'].nunique()

# Calculate the total budget

total_budget = school_data['budget'].sum()

# Convert the budget values to dollars

total_budget= '${:,.2f}'.format(total_budget)
# Calculate the average maths score 

average_maths_score = school_data_complete['maths_score'].mean()

# Calculate the average reading score

average_reading_score = school_data_complete['reading_score'].mean()

# Calculate the percentage of students with a passing maths score (50 or greater)

passing_maths = school_data_complete[(school_data_complete['maths_score'] >= 50)].shape[0]
percentage_passing_maths = (passing_maths / total_students) * 100

# Calculate the percentage of students with a passing reading score (50 or greater)

passing_reading = school_data_complete[(school_data_complete['reading_score'] >= 50)].shape[0]
percentage_passing_reading = (passing_reading / total_students) * 100

# Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

passing_both = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)].shape[0]
percentage_overall_passing = (passing_both / total_students) * 100



# Create a dataframe for the ket metrics
data = {
    'Total Schools': [total_schools],
    'Total Students': [total_students],
    'Total Budget': [total_budget],
    'Average Maths Score': [average_maths_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Maths': [percentage_passing_maths],
    '% Passing Reading': [percentage_passing_reading],
    '% Overall Passing': [percentage_overall_passing]
}

area_summary_df = pd.DataFrame(data)

# Display the DataFrame
print(area_summary_df.to_string(index=False))



 Total Schools  Total Students   Total Budget  Average Maths Score  Average Reading Score  % Passing Maths  % Passing Reading  % Overall Passing
            15           39170 $24,649,428.00            70.338192              69.980138        86.078632          84.426857          72.808272


 * **School Summary**

In [67]:

# Calculate the total student count for each school
total_students_per_school = school_data_complete.groupby('school_name')['Student ID'].count()

# Calculate the per student budget for each school
school_data['Per Student Budget'] = school_data['budget'] / school_data['size']

# Calculate the average test scores for each school
average_scores_per_school = school_data_complete.groupby('school_name')['maths_score', 'reading_score'].mean()

# Calculate the number of students passing math for each school
passing_math_per_school = school_data_complete[school_data_complete['maths_score'] >= 50].groupby('school_name')['Student ID'].count()

# Calculate the number of students passing reading for each school
passing_reading_per_school = school_data_complete[school_data_complete['reading_score'] >= 50].groupby('school_name')['Student ID'].count()

# Calculate the number of students passing both math and reading for each school
passing_both_per_school = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)].groupby('school_name')['Student ID'].count()

# Calculate the passing rates for each school
passing_rates_per_school = pd.DataFrame({
    '% Passing Maths': (passing_math_per_school / total_students_per_school) * 100,
    '% Passing Reading': (passing_reading_per_school / total_students_per_school) * 100,
    '% Overall Passing': (passing_both_per_school / total_students_per_school) * 100
})

# Convert the budget values to dollars
school_data['Total School Budget'] = school_data['budget'].map('${:,.2f}'.format)

# Convert the Per Student Budget in dollars
school_data['Per Student Budget'] = school_data['Per Student Budget'].map('${:,.2f}'.format)

# Create the per_school_summary DataFrame
per_school_summary = pd.DataFrame({
    'School Type': school_data.set_index('school_name')['type'],
    'Total Students': total_students_per_school,
    'Total School Budget': school_data.set_index('school_name')['Total School Budget'],
    'Per Student Budget': school_data.set_index('school_name')['Per Student Budget'],
    'Average Maths Score': average_scores_per_school['maths_score'],
    'Average Reading Score': average_scores_per_school['reading_score'],
    '% Passing Maths': passing_rates_per_school['% Passing Maths'],
    '% Passing Reading': passing_rates_per_school['% Passing Reading'],
    '% Overall Passing': passing_rates_per_school['% Overall Passing']
})

# Display the per_school_summary DataFrame in a tabular format

per_school_summary.reset_index(inplace=True)
print(per_school_summary.to_string(index=False))


          school_name School Type  Total Students Total School Budget Per Student Budget  Average Maths Score  Average Reading Score  % Passing Maths  % Passing Reading  % Overall Passing
   Bailey High School  Government            4976       $3,124,928.00            $628.00            72.352894              71.008842        91.639871          87.379421          80.084405
  Cabrera High School Independent            1858       $1,081,356.00            $582.00            71.657158              71.359526        90.850377          89.074273          80.785791
 Figueroa High School  Government            2949       $1,884,411.00            $639.00            68.698542              69.077993        81.654798          82.807731          67.650051
     Ford High School  Government            2739       $1,763,916.00            $644.00            69.091274              69.572472        82.438846          82.219788          67.469880
  Griffin High School Independent            1468         $9

/var/folders/82/73m8cjcn69lb5gvwvc4tnlq40000gn/T/ipykernel_37031/3353796532.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  average_scores_per_school = school_data_complete.groupby('school_name')['maths_score', 'reading_score'].mean()


## Top Performing Schools (By % Overall Passing)

In [51]:
# Sort the schools by '% Overall Passing' in descending order
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

# Display the top_schools DataFrame

print(top_schools.to_string(index=False))

          school_name School Type  Total Students Total School Budget Per Student Budget  Average Maths Score  Average Reading Score  % Passing Maths  % Passing Reading  % Overall Passing
  Griffin High School Independent            1468         $917,500.00            $625.00            71.788147              71.245232        91.212534          88.487738          81.335150
  Cabrera High School Independent            1858       $1,081,356.00            $582.00            71.657158              71.359526        90.850377          89.074273          80.785791
   Bailey High School  Government            4976       $3,124,928.00            $628.00            72.352894              71.008842        91.639871          87.379421          80.084405
   Wright High School Independent            1800       $1,049,400.00            $583.00            72.047222              70.969444        91.777778          86.666667          79.722222
Rodriguez High School  Government            3999       $2,5

## Bottom Performing Schools (By % Overall Passing)

In [52]:
# Sort the schools by '% Overall Passing' in ascending order and select the first 5 rows
bottom_schools = per_school_summary.sort_values(by='% Overall Passing').head(5)

# Display the bottom_schools DataFrame
print(bottom_schools.to_string(index=False))

          school_name School Type  Total Students Total School Budget Per Student Budget  Average Maths Score  Average Reading Score  % Passing Maths  % Passing Reading  % Overall Passing
Hernandez High School  Government            4635       $3,022,020.00            $652.00            68.874865              69.186408        80.949299          81.877023          66.364617
    Huang High School  Government            2917       $1,910,635.00            $655.00            68.935207              68.910525        81.693521          81.453548          66.712376
  Johnson High School  Government            4761       $3,094,650.00            $650.00            68.843100              69.039277        82.062592          81.978576          67.191766
   Wilson High School Independent            2283       $1,319,574.00            $578.00            69.170828              68.876916        82.785808          81.296540          67.455103
     Ford High School  Government            2739       $1,7

## Maths Scores by Year

In [53]:

# Using the groupby to calculate maths score for students of each year level at each school
maths_scores_by_year = school_data_complete.groupby(['school_name', 'year'])['maths_score'].mean()

# Convert the Series to a DataFrame
maths_scores_by_year = maths_scores_by_year.reset_index()

# Pivot the DataFrame to have years as columns
maths_scores_by_year = maths_scores_by_year.pivot(index='school_name', columns='year', values='maths_score')

# Rename the columns to represent the year levels
maths_scores_by_year.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']

# Display the maths_scores_by_year DataFrame
maths_scores_by_year.reset_index(inplace=True)
print(maths_scores_by_year.to_string(index=False))


          school_name    Year 9   Year 10   Year 11   Year 12
   Bailey High School 72.493827 71.897498 72.374900 72.675097
  Cabrera High School 72.321970 72.437768 71.008299 70.604712
 Figueroa High School 68.477804 68.331586 68.811001 69.325282
     Ford High School 69.021609 69.387006 69.248862 68.617811
  Griffin High School 72.789731 71.093596 71.692521 71.469178
Hernandez High School 68.586831 68.867156 69.154412 68.985075
   Holden High School 70.543307 75.105263 71.640777 73.409639
    Huang High School 69.081754 68.533246 69.431345 68.639316
  Johnson High School 69.469286 67.990220 68.637730 69.287393
     Pena High School 71.996364 72.396000 72.523438 71.187845
Rodriguez High School 71.940722 71.779808 72.364811 72.154626
  Shelton High School 72.932075 72.506696 70.097087 72.331536
   Thomas High School 69.234273 70.057007 69.657831 69.369822
   Wilson High School 69.212361 69.455446 68.378965 69.787472
   Wright High School 71.741176 72.179012 73.275862 70.848238


## Reading Score by Year

In [55]:
# Using the groupby to calculate reading score for students of each year level at each school
reading_scores_by_year = school_data_complete.groupby(['school_name', 'year'])['reading_score'].mean()

# Convert the Series to a DataFrame
reading_scores_by_year = reading_scores_by_year.reset_index()

# Pivot the DataFrame to have years as columns
reading_scores_by_year = reading_scores_by_year.pivot(index='school_name', columns='year', values='reading_score')

# Rename the columns to represent the year levels
reading_scores_by_year.columns = ['Year 9', 'Year 10', 'Year 11', 'Year 12']

# Display the maths_scores_by_year DataFrame
reading_scores_by_year.reset_index(inplace=True)
print(reading_scores_by_year.to_string(index=False))

          school_name    Year 9   Year 10   Year 11   Year 12
   Bailey High School 70.901920 70.848265 70.317346 72.195525
  Cabrera High School 71.172348 71.328326 71.201245 71.856021
 Figueroa High School 70.261682 67.677588 69.152327 69.082126
     Ford High School 69.615846 68.988701 70.735964 68.849722
  Griffin High School 72.026895 70.746305 72.385042 69.434932
Hernandez High School 68.477569 70.621842 68.418199 69.244136
   Holden High School 71.598425 71.096491 73.310680 70.481928
    Huang High School 68.670616 69.516297 68.740638 68.671795
  Johnson High School 68.719286 69.295029 69.969115 67.992521
     Pena High School 70.949091 72.324000 71.703125 71.513812
Rodriguez High School 70.902921 70.137500 71.424453 71.414449
  Shelton High School 70.715094 69.879464 71.150485 69.070081
   Thomas High School 69.672451 69.741093 70.749398 68.730769
   Wilson High School 68.683043 68.412541 68.796327 69.888143
   Wright High School 71.823529 71.386831 70.475862 69.821138


## Scores by School Spending

In [64]:
# Convert 'Per Student Budget' column to numeric by removing non-numeric characters and converting to float
school_data['Per Student Budget'] = school_data['Per Student Budget'].replace('[\$,]', '', regex=True).astype(float)

# Define the spending bins and labels
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ['<$585', '$585 - $630', '$630 - $645', '$645 - $680']

# Group schools into spending ranges using cut
school_data['Spending Ranges (Per Student)'] = pd.cut(school_data['Per Student Budget'], bins = spending_bins, labels = spending_labels)


# Calculate the average maths score, average reading score, % passing maths, % passing reading,
# and overall passing rate for each spending range

spending_math_scores = school_data_complete.groupby(["Spending Ranges (Per Student)"])["Average Maths Score"].mean()
spending_reading_scores = school_data_complete.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_data_complete.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_data_complete.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_data_complete.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()


# Create a DataFrame to hold the results
spending_summary = pd.DataFrame({
    'Average Maths Score': spending_math_scores,
    'Average Reading Score': spending_reading_scores,
    '% Passing Maths': spending_passing_math,
    '% Passing Reading': spending_passing_reading,
    '% Overall Passing': overall_passing_spending
})

# Display the spending_summary DataFrame
spending_summary.reset_index(inplace=True)
print(spending_summary.to_string(index=False))

KeyError: 'Column not found: Average Maths Score'

* Perform the same operations as above, based on school size.

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
